# Plot Enemy Sprites

Run `iridisalpha.prg` with the following trace:

```
log on                                            
logname "IridisAlphaTitleEnemySpriteMovements.txt"
tr exec 6b01                                      
command 1 "mem d000 d010; mem 07f8 07ff"          
```

In [1]:
colors = {
"00":"c64_black",  
"01":"c64_white",  
"02":"c64_red",    
"03":"c64_cyan",   
"04":"c64_purple", 
"05":"c64_green",  
"06":"c64_blue",   
"07":"c64_yellow", 
"08":"c64_orange", 
"09":"c64_brown",  
"0a":"c64_ltred",  
"0b":"c64_gray1",  
"0c":"c64_lightgray",  
"0d":"c64_ltgreen",
"0e":"c64_ltblue", 
"0f":"c64_gray3",  
}
c64_to_rgb = {
"c64_black": "#000000",     
"c64_white": "#ffffff",     
"c64_red": "#880000",       
"c64_cyan":  "#aaffee",     
"c64_purple": "#cc44cc",    
"c64_green": "#00cc55",     
"c64_blue":  "#0000aa",     
"c64_yellow":  "#eeee77",   
"c64_orange":  "#dd8855",   
"c64_brown": "#664400",     
"c64_ltred": "#ff7777",  
"c64_gray1":  "#333333",    
"c64_lightgray": "#bbbbbb", 
"c64_ltgreen": "#aaff66",
"c64_ltblue":  "#0088ff",
"c64_gray3": "#bbbbbb",                             
}
hex_to_rgb = {
"f0": "#000000",     
"f1": "#ffffff",     
"f2": "#880000",       
"f3":  "#aaffee",     
"f4": "#cc44cc",    
"f5": "#00cc55",     
"f6":  "#0000aa",     
"f7":  "#eeee77",   
"f8":  "#dd8855",   
"f9": "#664400",     
"fa": "#ff7777",  
"fb":  "#333333",    
"fc": "#bbbbbb", 
"fd": "#aaff66",
"fe":  "#0088ff",
"ff": "#bbbbbb",                             
}


In [2]:
from sprite_map import sprite_map 


We need this to identify if the sprite is appropriate for the level. This works around the bug where sprites from a previous game can appear in the current level.

In [3]:
"""
Pull out all the level data from the source files.
"""
files = ["../iridisalpha/src/level_data/level_data.asm",
         "../iridisalpha/src/level_data/level_data2.asm"]
d = {}
level_data = {}
ch = None
ck = None
for f in files:
    ld = open(f, 'r')
    while True:
        l = ld.readline()
        if not l:
            break
        if "pieceOfPlanet" in l:
            continue
        if l[:2].strip():
            if ch and d:
                level_data[ch] = d
            ch = l.split()[0].strip()
            d = {}
            continue

        k = l[10:17]
        if "Byte" in k:
            ck = k
            continue

        if "BYTE" in l:
            v = l[14:].strip().split(',')[0].replace('<','')
            d[ck] = v

# Drop any spurious entries
level_data = {k:c for k,c in level_data.items() if len(c) == 30}
level_data["lickerShipWaveData"]

{'Byte 0 ': '$0B',
 'Byte 2 ': 'LICKERSHIP',
 'Byte 3 ': '$04',
 'Byte 5 ': 'LICKERSHIP_INV',
 'Byte 6 ': '$00',
 'Byte 8 ': 'nullPtr',
 'Byte 10': 'nullPtr',
 'Byte 11': '$00',
 'Byte 13': 'nullPtr',
 'Byte 14': '$00',
 'Byte 15': '$20',
 'Byte 17': 'defaultExplosion',
 'Byte 18': '$00',
 'Byte 19': '$00',
 'Byte 20': '$02',
 'Byte 21': '$02',
 'Byte 22': '$01',
 'Byte 23': '$01',
 'Byte 25': 'nullPtr',
 'Byte 27': 'nullPtr',
 'Byte 29': 'nullPtr',
 'Byte 31': 'lickerShipWaveData',
 'Byte 32': '$00',
 'Byte 33': '$00',
 'Byte 34': '$00',
 'Byte 35': '$02',
 'Byte 36': '$00',
 'Byte 37': '$00',
 'Byte 38': '$00',
 'Byte 39': '$00'}

In [13]:
def getLinkedLevels(l,k):
    if k not in level_data:
        print("ERROR:", k)
        return []
    v = level_data[k]
    b1 = v["Byte 17"]
    b2 = v["Byte 29"]
    b3 = v["Byte 31"]
    for b in [b1,b2,b3]:
        if b in l or b in ['nullPtr', 'defaultExplosion','spinningRings']:
            continue
        l += [b]
        l = getLinkedLevels(l,b)
    return l

linked_data = {}
for p in range(1,6):
    for l in range(0,21):
        lname = f"planet{p}Level{l}Data"
        d = getLinkedLevels([lname], lname)
        linked_data[f"{p}_{l}"] = d

level_sprites = {}
for k,v in linked_data.items():
    sprites = []
    for l in v:
        sprite = level_data[l]["Byte 2 "]
        sprite = re.sub(r"[0-9]$","",sprite)
        sprites += [sprite]
    level_sprites[k] = list(set(sprites))
level_sprites

ERROR: planet1Level0Data
ERROR: planet2Level0Data
ERROR: planet3Level0Data
ERROR: $50
ERROR: planet4Level0Data
ERROR: planet5Level0Data
ERROR: planet5Level16Data
ERROR: planet5Level19Data


{'1_0': [],
 '1_1': ['FLYING_SAUCER'],
 '1_2': ['BOUNCY_RING'],
 '1_3': ['FLYING_DOT', 'LICKERSHIP'],
 '1_4': ['FLYING_FLOWCHART', 'FLYING_TRIANGLE'],
 '1_5': ['BALLOON', 'SMALL_BALL'],
 '1_6': ['SPINNING_RING', 'BIRD'],
 '1_7': ['FLAG_BAR', 'FLAG_BAR+$0'],
 '1_8': ['TEARDROP_EXPLOSION'],
 '1_9': ['CAMEL', 'WINGBALL', 'WINGED_BALL'],
 '1_10': ['CAMEL'],
 '1_11': ['CAMEL', 'GILBY_AIRBORNE_LEFT'],
 '1_12': ['CAMEL'],
 '1_13': ['CAMEL', 'LLAMA', 'BUBBLE'],
 '1_14': ['TEARDROP_EXPLOSION'],
 '1_15': ['TEARDROP_EXPLOSION', 'LLAMA', 'LICKERSHIP'],
 '1_16': ['QBERT_SQUARES'],
 '1_17': ['GILBY_AIRBORNE_LEFT', 'CAMEL', 'BOUNCY_RING'],
 '1_18': ['LAND_GILBY', 'GILBY_AIRBORNE_RIGHT'],
 '1_19': ['STARSHIP'],
 '1_20': ['COPTIC_CROSS'],
 '2_0': [],
 '2_1': ['LITTLE_DART'],
 '2_2': ['CAMEL', 'FLYING_COCK', 'LICKERSHIP'],
 '2_3': ['FLYING_COCK_RIGHT', 'CAMEL', 'LICKERSHIP'],
 '2_4': ['TEARDROP_EXPLOSION', 'CAMEL'],
 '2_5': ['LICKER_SHIP', 'LICKERSHIP'],
 '2_6': ['SPINNING_RING', 'CAMEL', 'LICKERSHIP'],

Extract the x and y positions used foreach of the enemy sprites in attract mode.

Modify the routine for selecting random planets as follows, so we catch them all.
```
planetSelector       .BYTE $00                             
levelSelector       .BYTE $00                              
;-------------------------------------------------------   
; SelectRandomPlanetsForAttractMode                        
;-------------------------------------------------------   
SelectRandomPlanetsForAttractMode                          
                                                           
        LDA levelSelector                                  
        ; Select 9 random numbers between 0 and 15         
        LDX #$09                                           
b7EC5   STA currentLevelInTopPlanets,X                     
        DEX                                                
        BPL b7EC5                                          
                                                           
        INC levelSelector                                  
        LDA levelSelector                                  
        CMP #$14                                           
        BNE SetRandomPlanet                                
        LDA #$00                                           
        STA levelSelector                                  
                                                           
        INC planetSelector                                 
        LDA planetSelector                                 
        CMP #$05                                           
        BNE SetRandomPlanet                                
        LDA #$00                                           
        STA planetSelector                                 
                                                           
SetRandomPlanet                                            
        ; Select a random planet between 0 and 3           
        LDA planetSelector                                 
        STA topPlanetPointerIndex                          
                                                           
        ; Select a random planet between 0 and 3           
        STA bottomPlanetPointerIndex                       
                                                           
ReturnFromRandomPlanet                                     
        RTS                                                

```

In [4]:
import gzip
log_file = "IridisAlphaTitleEnemySpriteMovements.txt.gz"
input_file = gzip.open(log_file,'rt')

movements = [ [] for _ in range(8) ]
for l in input_file.readlines():
    # the x and y co-ords for each sprite
    if "C:d000" in l:
        s = l[9:59]
        s = s.replace(' ',',').replace(',,',',')
        s = s.split(',')
        co_ords = [(int(s[i],16), int(s[i+1],16)) for i in range(0,len(s),2)]
        continue

    # sprite_colors
    if "C:d025" in l:
        s = l[9:40]
        s = s.replace(' ',',').replace(',,',',')
        sprite_colors_array = s.split(',')
        multi_color0 = sprite_colors_array[0]
        multi_color1 = sprite_colors_array[1]
        sprite_colors = sprite_colors_array[2:]
        continue

    # the most significant bit for each sprite's x co-ord
    if "C:d010" in l:
        msb = int(l[9:11],16)
        continue

    # The current sprite value
    if "C:07f8" in l:
        s = l[9:33]
        s = s.replace(' ',',').replace(',,',',')
        sprite_values = s.split(',')
        continue
        
    # current planet
    # change back to 78b0 in unmodified rom
    if "C:78b3" in l:
        planet = int(l[9:11],16)
        continue

    # current level
    # 49c6 in unmodified rom
    if not "C:49c9" in l:
        continue
    s = l[9:33]
    s = s.replace(' ',',').replace(',,',',')
    levels = s.split(',')
    level = int(levels[planet],16)

    # Add the most significant bit to each sprite's x value
    for i,v in enumerate([1,2,4,8,16,32,64,128]):
        if msb & v:
            x,y = co_ords[i]
            co_ords[i] = (x+256,y)

    for i, spr in enumerate(sprite_values):
        movements[i] += [(sprite_values[i], co_ords[i], (planet,level),
                         (multi_color0,multi_color1,sprite_colors[i]))]


Load in the sprite data

In [5]:
import re
sprites_files = ["../iridisalpha/src/graphics/sprites.asm",
                 "../iridisalpha/src/graphics/enemy_sprites.asm"]
sprites_data = {}
for sprites_file in sprites_files:
    input_file = open(sprites_file,'r')
    sprite_data = []
    for l in input_file.readlines():
        if "SPRITE" in l:
            if sprite_data:
                sprites_data[sprite_name] = sprite_data
            sprite_name = l[18:20].strip().lower()
            sprite_data = []
            continue

        m = re.findall(r"[0-1]{24}",l)
        if not m:
            continue
        bits = m[0]
        sprite_line = []
        for i in range(0,23,2):
            bitpair = bits[i:i+2]
            sprite_line += [bitpair]
            sprite_line += [bitpair]
        sprite_data += [sprite_line]
    if sprite_data:
        sprites_data[sprite_name] = sprite_data


In [6]:
"""
   BIT PAIR                           DESCRIPTION                          
 ------------------------------------------------------------------------- 
     00        TRANSPARENT, SCREEN COLOR                                   
     01        SPRITE MULTI-COLOR REGISTER #0 (53285) ($D025)              
     10        SPRITE COLOR REGISTER                                       
     11        SPRITE MULTI-COLOR REGISTER #I (53286) ($D026)              
"""
black_list = [
'LAND_GILBY', 'NO_SPRITE',
'BLANK_SPRITE', 'GILBY_AIRBORNE_LEFT', 'EXPLOSION_END',
'EXPLOSION_MIDDLE','EXPLOSION_START', 'GILBY_AIRBORNE_RIGHT', 
'LASER_BULLET', 'GILBY_TAKING_OFF','SPINNING_RING','GILBY_AIRBORNE_TURNING',
'TEARDROP_EXPLOSION'
]


In [7]:
from PIL import Image, ImageColor
cols=40
rows=25
bits=8
width = cols*bits
height = rows*bits

def paintSprite(origin, pixels, sprite, colors):
    multicol0, multicol1,color = colors
    colormap = {
        "01": multicol0,
        "10": color,
        "11": multicol1,
    }
    
    x_o, y_o = origin
    if sprite not in sprites_data:
        print(sprite)
        return
    bit_array = sprites_data[sprite]
    for y, l in enumerate(bit_array):
        for x,bit in enumerate(l):
            if bit == "00":
                continue
            #color = c64_to_rgb[colors[color_ram[r][c]]]
            if (x_o+x > width-1) or (y_o+y > height - 1):
                continue
            pixel_color = ImageColor.getrgb(hex_to_rgb[colormap[bit]])
            pixels[x_o+x, y_o+y] = pixel_color



Write out the sprite movement sequences to image files

In [8]:
surfaces = {
    0: Image.open("../src/planets/planet1Surface.png"),
    1: Image.open("../src/planets/planet2Surface.png"),
    2: Image.open("../src/planets/planet3Surface.png"),
    3: Image.open("../src/planets/planet4Surface.png"),
    4: Image.open("../src/planets/planet5Surface.png"),
}
#img = Image.open(file_name)
#background = Image.new('RGB', (504,312), "black")
#background.paste(img, (60,20))



In [18]:
import os

from collections import defaultdict

img = Image.new( 'RGB', (width,height), "black") # create a new black image
pixels = img.load() # create the pixel map

max_files = 4
files_written = defaultdict(lambda: 0)
prev_sprite = ""
prev_planet = ""
prev_level = ""
for j, mvt in enumerate(movements[1:6]):
    for i,(sprite, origin, (planet,level), colors) in enumerate(mvt):
        if sprite not in sprite_map:
            continue
        sprite_name = sprite_map[sprite]
        level_name = f"{planet+1}_{level+1}"
        if files_written[level_name] > (max_files - 1):
            continue
        #if sprite_name in black_list:
        #    continue
        if sprite_name not in level_sprites[level_name]:
            continue
        if (prev_level and level_name != prev_level):
            length = files_written[prev_level]
            file_name = f"../src/level_data/images/movement_{prev_level}_{length}.png"
            if length < max_files and not os.path.isfile(file_name):
                #im2 = img.crop((0,min_y,width,max_y))
                #img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
                img.paste(surfaces[prev_planet], (0,130))
                img.save(file_name)
                files_written[prev_level] += 1

            img = Image.new( 'RGB', (width,height), "black")
            pixels = img.load()

        prev_level = level_name
        prev_planet = planet
        paintSprite(origin, pixels, sprite, colors)

file_name = f"../src/level_data/images/movement_{prev_level}_{length}.png"
img.paste(surfaces[prev_planet], (0,130))
img.save(file_name)



Create an Appendix tabulating the data for each level and with 2 images representing enemy movement

In [40]:
#all_fields = list(range(0,30))
sprite_detail = [0,1,2,3]
basic_movement = [4,12,13,14,15]
enemy_behaviour = [10,16,17,24,25]
pointer_data = [5,11,18,19,20,21]

header = """
\\begin{figure}[H]
  {
  \\setlength{\\tabcolsep}{3.0pt}
  \\setlength\\cmidrulewidth{\\heavyrulewidth} % Make cmidrule = 
  \\begin{adjustbox}{width=12cm}
"""

footer = """
  \\end{adjustbox}

  }\\caption*{%CAPTION%.}
\\end{figure}
"""

byte_descriptions = {
"Level" : "",
"Byte 0":"Index into array for sprite color",                                   
"Byte 1":"Sprite values for the attack ship on the upper planet",          
"Byte 2":"First and last sprite value for the attack ship on the upper planet",           
"Byte 3":"The animation frame rate for the attack ship.",                       
"Byte 4":"First sprite value for the attack ship on the lower planet",          
"Byte 5":"First and last sprite value for the attack ship on the lower planet",           
"Byte 6":"Rate at which to switch to alternate enemy mode.",                    
"Byte 7":"Lo Ptr for alternate enemy mode",                                     
"Byte 8":"Pointer for alternate enemy mode",                                     
"Byte 9":"Unused Lo Ptr to an arbitrary run of Bytes 18-21.",
"Byte 10":"Unused Pointer to an arbitrary run of Bytes 18-21.",
"Byte 11":"Unused Rate limit for use of Bytes 9 and 10.",              
"Byte 12":"Unused Byte",                                                     
"Byte 13":"Unused Bytes",                                                     
"Byte 14":"Controls the rate at which new enemies are added.",
"Byte 15":"Update rate for switching to the level data in Bytes 16/17",                                        
"Byte 16":"Lo Ptr to wave data we switch to periodically.",               
"Byte 17":"Pointer to wave data we switch to periodically.",               
"Byte 18":"X Pos movement for attack ship.",                                    
"Byte 19":"Y Pos movement pattern for attack ship.",                            
"Byte 20":"X Pos Frame Rate for Attack ship.",                                  
"Byte 21":"Y Pos Frame Rate for Attack ship.",                                  
"Byte 22":"Stickiness factor, does the enemy stick to the player",              
"Byte 23":"Does the enemy gravitate quickly toward the player when its hit?",   
"Byte 24":"Lo Ptr for second wave of attack ships.",                            
"Byte 25":"Pointer for second wave of attack ships.",                            
"Byte 26":"Lo Ptr for third wave of attack ships.",                             
"Byte 27":"Pointer for third wave of attack ships.",                             
"Byte 28":"Lo Ptr for wave to switch to when hit by bullet.",                                    
"Byte 29":"Pointer for wave to switch to when hit by bullet.",
"Byte 30":"Lo Ptr for wave to switch to after colliding with gilby.",
"Byte 31":"Pointer for  wave to switch to after colliding with gilby.",
"Byte 32":"Lo Ptr for fourth wave of attack ships.",                            
"Byte 33":"Pointer for fourth wave of attack ships.",                            
"Byte 34":"Points for hitting the enemy.",                                      
"Byte 35":"Energy increase multiplier for hitting an enemy.",
"Byte 36":"Is the ship a spinning ring, i.e. does it allow the gilby to warp?", 
"Byte 37":"Number of waves in data.",                                           
"Byte 38":"Number of ships in wave.",                                           
"Byte 39":"Unused byte.",                                               
}

adl = "\\addlinespace"
leg = "\\multicolumn{%len%}{@{}l@{}}{%BYTE%}\\\\"



In [20]:
import pandas as pd
from tabulate import tabulate
from texttable import Texttable
import latextable


## Create an appendix file with the images and level data

In [41]:
import os

mks = {
"Byte 2 ":"Byte 1-2",
"Byte 5 ":"Byte 4-5",
"Byte 8 ":"Byte 7-8",
"Byte 10":"Byte 9-10",
"Byte 13":"Byte 12-13",
"Byte 17":"Byte 16-17",
"Byte 25":"Byte 24-25",
"Byte 27":"Byte 26-27",
"Byte 29":"Byte 28-29",
"Byte 31":"Byte 30-31",
"Byte 33":"Byte 32-33",
}
planet_names = ["","Sheep", "Tech", "Brick", "Mushroom", "Om"]
output_file = open("../src/level_data/level_data_appendix.tex", 'w')
for p in range(1,6):
    for l in range(1,21):
        lname = f"planet{p}Level{l}Data"
        if lname not in level_data:
            continue
        d = level_data[lname]
        t = []
        for k,v in d.items():
            if k == "Byte 32":
                continue
            tk = k if k not in mks else mks[k]
            t += [[tk,v,byte_descriptions[k.strip()]]]

        h = f"""
\\clearpage
\\subsubsection{{{planet_names[p]} Planet - Level {l} Data}}
"""
        output_file.write(h)
        
        screenshots = f"""
\\begin{{figure}}[H]
    \\centering
    \\foreach \\l in {{0,...,1}}
    {{
      \\includegraphics[width=6cm]{{level_data/images/movement_{p}_{l}_\l.png}}%
    }}%
\caption*{{Sample enemy movement for {planet_names[p]} planet, level {l}.}}
\end{{figure}}
"""
        fname = f"../src/level_data/images/movement_{p}_{l}_1.png"
        if os.path.isfile(fname):
            output_file.write(screenshots+'\n')

        output_file.write(header+'\n')

        ltable = tabulate(t, headers=["Byte","Value","Description"], 
                          tablefmt='latex_booktabs')
        output_file.write(ltable+'\n')
        output_file.write(footer.replace("%CAPTION%", 
                                         f"{planet_names[p]} Planet - Level {l}\n"))

output_file.close()